### RAG Pipelines : Data Ingestion to Vector DB pipeline

In [2]:
%pip install langchain-text-splitters
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


c:\Users\Lenovo\OneDrive\Desktop\QueryBased_PDF_Summarizer\.venv\Scripts\python.exe: No module named pip


Note: you may need to restart the kernel to use updated packages.


In [3]:
### REAL ALL THE PDFS INSIDE DIRECTORY
from pathlib import Path

def process_all_pdfs(pdf_directory):
    all_documents=[]
    pdf_dir = Path(pdf_directory)

    #finding pdf recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing:{pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Extract author from the first page's text
            first_page_text = documents[0].page_content
            ##found_author = extract_author_from_text(first_page_text)

            #ADD source info to meta data  
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type']='pdf'
                doc.metadata['author'] = doc.metadata.get('author', 'Unknown Author') ##after using llm just write here found_author
                doc.metadata['creation_date'] = doc.metadata.get('creationDate', 'Unknown Date')
               


            all_documents.extend(documents)
            print(f"loaded {len(documents)} pages")

        except Exception as e:
            print(f" Error :{e}")

    print(f"\nTotal documents loaded:{len(all_documents)}")        
    return all_documents

all_pdf_documents = process_all_pdfs("../data") 

Found 2 PDF files to process

Processing:AgenticAI.pdf
loaded 25 pages

Processing:Exploring_LLMs.pdf
loaded 24 pages

Total documents loaded:49


In [4]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.24; modified using iText® Core 7.2.4 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-01-28T21:29:41+05:30', 'moddate': '2025-01-29T10:28:02-05:00', 'ieee article id': '10849561', 'trapped': 'False', 'ieee issue id': '10820123', 'subject': 'IEEE Access;2025;13; ;10.1109/ACCESS.2025.3532853', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'ieee publication id': '6287639', 'title': 'Agentic AI: Autonomous Intelligence for Complex Goals&#x2014;A Comprehensive Survey', 'source': '..\\data\\pdf_files\\AgenticAI.pdf', 'total_pages': 25, 'page': 0, 'page_label': '18912', 'source_file': 'AgenticAI.pdf', 'file_type': 'pdf', 'author': 'Unknown Author', 'creation_date': 'Unknown Date'}, page_content='Received 1 January 2025, accepted 19 January 2025, date of publication 22 January 2025, date of current version 29 January 2025.\nDigital Object

In [5]:
## chunks 

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """SPlit documents into smaller chunks for Rag"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function = len,
        separators=["\n\n","\n"," ",""] # "\n\n"two consecutive newline characters
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    #show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    

    return split_docs

chunks  = split_documents(all_pdf_documents)
chunks

Split 49 documents into 353 chunks

Example chunk:
Content: Received 1 January 2025, accepted 19 January 2025, date of publication 22 January 2025, date of current version 29 January 2025.
Digital Object Identifier 10.1 109/ACCESS.2025.3532853
Agentic AI: Auto...
Metadata: {'producer': 'pdfTeX-1.40.24; modified using iText® Core 7.2.4 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-01-28T21:29:41+05:30', 'moddate': '2025-01-29T10:28:02-05:00', 'ieee article id': '10849561', 'trapped': 'False', 'ieee issue id': '10820123', 'subject': 'IEEE Access;2025;13; ;10.1109/ACCESS.2025.3532853', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'ieee publication id': '6287639', 'title': 'Agentic AI: Autonomous Intelligence for Complex Goals&#x2014;A Comprehensive Survey', 'source': '..\\data\\pdf_files\\AgenticAI.pdf', 'total_pages': 25, 'page': 0, 'page_label': '18912', 'source_file': 'A

[Document(metadata={'producer': 'pdfTeX-1.40.24; modified using iText® Core 7.2.4 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-01-28T21:29:41+05:30', 'moddate': '2025-01-29T10:28:02-05:00', 'ieee article id': '10849561', 'trapped': 'False', 'ieee issue id': '10820123', 'subject': 'IEEE Access;2025;13; ;10.1109/ACCESS.2025.3532853', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'ieee publication id': '6287639', 'title': 'Agentic AI: Autonomous Intelligence for Complex Goals&#x2014;A Comprehensive Survey', 'source': '..\\data\\pdf_files\\AgenticAI.pdf', 'total_pages': 25, 'page': 0, 'page_label': '18912', 'source_file': 'AgenticAI.pdf', 'file_type': 'pdf', 'author': 'Unknown Author', 'creation_date': 'Unknown Date'}, page_content='Received 1 January 2025, accepted 19 January 2025, date of publication 22 January 2025, date of current version 29 January 2025.\nDigital Object

## Embedding  and VectorStoredb


In [6]:
import numpy as np 
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid  ## every recoed we enter into the vectordb has some record id 
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    """Handles Document embedding  generation using SentenceTransformer"""
    def __init__(self, model_name: str ="all-MiniLM-L6-v2"):
        """
        intitalize the embedding manager 
        Args:
           model_name:HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()


    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model:{self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully.Embedding dimnesion :{ self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model{self.model_name}:{e}")
            raise


    def  generate_embeddings(self,texts: List[str]) ->np.ndarray:
        """ generate embeddings for a list of texts
        Args:
            texts: list of text strings to embed
        Returns:
            numpy array of embeddings with shape (len(texts),embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape:{embeddings.shape}")
        return embeddings 
    


    ## intitalise the embedding manager 
embedding_manager = EmbeddingManager()
embedding_manager 


Loading embedding model:all-MiniLM-L6-v2
Model loaded successfully.Embedding dimnesion :384


### VectorStore


In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    def __init__(self,collection_name:str ="pdf_documents", persist_directory: str = "../data/vector_store"):
        """Initialize the vector store 
        Args:
            collection_name: Name of the chromaDB collection
            presist_directory: Directory to presist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self .client = None
        self.collection = None
        self._initialize_store()
    

    def _initialize_store(self):
        """Initialise chromaDB client and collection"""
        try:
            #create presistent chromaDB client 
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            #get or create collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description":"PDF document embeddingd for RAG"}
            )

            print(f"Vector store intialised. COllection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initalizing vector store:{e}")
            raise 


    def  add_documents(self,documents: List[Any],embeddings : np.ndarray):
        """
        Add documents and their embeddings to their vector store 

        Args:
            documents : List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents)!=len(embeddings):
            raise ValueError("Number of documents must number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")

        #prepare data for ChromaDB
        ids =[]
        metadatas =[]
        documents_text =[]
        embeddings_list =[]


        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)


            # prepare metadata 
            metadata= dict(doc.metadata)
            metadata['doc_index']=i
            metadata['content_length']=len(doc.page_content)
            metadatas.append(metadata)

            #Document content 
            documents_text.append(doc.page_content)

           #Embedding
            embeddings_list.append(embedding.tolist())

        #ADD to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings= embeddings_list,
                metadatas = metadatas,
                documents = documents_text
            )    
            print(f"Sucessfully added {len(documents)} documents to vector store")
            print(f"Total documnets in collection : {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store:{e}")
            raise


vectorstore = VectorStore()
vectorstore


Vector store intialised. COllection: pdf_documents
Existing documents in collection: 1059


In [9]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.24; modified using iText® Core 7.2.4 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-01-28T21:29:41+05:30', 'moddate': '2025-01-29T10:28:02-05:00', 'ieee article id': '10849561', 'trapped': 'False', 'ieee issue id': '10820123', 'subject': 'IEEE Access;2025;13; ;10.1109/ACCESS.2025.3532853', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'ieee publication id': '6287639', 'title': 'Agentic AI: Autonomous Intelligence for Complex Goals&#x2014;A Comprehensive Survey', 'source': '..\\data\\pdf_files\\AgenticAI.pdf', 'total_pages': 25, 'page': 0, 'page_label': '18912', 'source_file': 'AgenticAI.pdf', 'file_type': 'pdf', 'author': 'Unknown Author', 'creation_date': 'Unknown Date'}, page_content='Received 1 January 2025, accepted 19 January 2025, date of publication 22 January 2025, date of current version 29 January 2025.\nDigital Object

In [10]:
### convert the text to embeddings
texts=[doc.page_content for doc in chunks]


## generate the embeddings
embeddings = embedding_manager.generate_embeddings(texts)

##store in the vector database
vectorstore.add_documents(chunks,embeddings)


Generating embeddings for 353 texts...


Batches: 100%|██████████| 12/12 [00:12<00:00,  1.08s/it]


Generated embeddings with shape:(353, 384)
Adding 353 documents to vector store...
Sucessfully added 353 documents to vector store
Total documnets in collection : 1412


### Retriever Pipleline From VectorStore

In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store.
       retriever is built on vectorStore , its like a interface
    """
    def __init__(self,vector_store:VectorStore,embedding_manager:EmbeddingManager):
        """
        Initialise the retriever
        Args:
            vector_store:Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """

        self.vector_store = vector_store
        self.embedding_manager = embedding_manager 

    def retrieve(self,query: str,top_k: int = 5,score_threshold: float =0.0) -> List[Dict[str,Any]]:
        """
        Retrieve relevant documnets for a query
        Args:
            query:The search query
            top_k: Number of top results to return 
            scope_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries contauining retrieved documents and metadata 
        """

        print(f"Retrieveing documents for query: '{query}'")
        print(f"Top K :{top_k},Score threshold :{score_threshold}")

        #Generate query embedding 
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        #Search in vectorStore
        try:
            results = self.vector_store.collection.query(
                query_embeddings =[query_embedding.tolist()],
                n_results = top_k
            )

            #process results 
            retrieved_docs =[]

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]


                for i, (doc_id,document,metadata,distance) in enumerate(zip(ids,documents,metadatas,distances)):
                    similarity_score  = 1- distance

                    if similarity_score >= score_threshold:
                       retrieved_docs.append({
                           'id': doc_id,
                           'content':document,
                           'metadata': metadata,
                           'similarity_score' : similarity_score,
                           'distance' :distance,
                           'rank': i+1
                       })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")

            else:
                print("No document found")
            

            return retrieved_docs
        
        except Exception as e:
            print(f"Error during retrieval : {e}")
            return []
        

rag_retriever= RAGRetriever(vectorstore,embedding_manager)


In [12]:
rag_retriever

In [13]:
rag_retriever.retrieve("architectural approach in agentic ai ")

Retrieveing documents for query: 'architectural approach in agentic ai '
Top K :5,Score threshold :0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.17it/s]

Generated embeddings with shape:(1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_9042a5cd_47',
  'content': 'D. B. Acharya et al.: Agentic AI: Autonomous Intelligence for Complex Goals—A Comprehensive Survey\nFIGURE 3. Overview of Agentic AI Development Methodologies, including Architectural Approaches, Learning\nParadigms, Training Techniques, and Tools.\n18920 VOLUME 13, 2025',
  'metadata': {'title': 'Agentic AI: Autonomous Intelligence for Complex Goals&#x2014;A Comprehensive Survey',
   'subject': 'IEEE Access;2025;13; ;10.1109/ACCESS.2025.3532853',
   'trapped': 'False',
   'creationdate': '2025-01-28T21:29:41+05:30',
   'creator': 'LaTeX with hyperref',
   'producer': 'pdfTeX-1.40.24; modified using iText® Core 7.2.4 (AGPL version) ©2000-2022 iText Group NV',
   'page': 8,
   'page_label': '18920',
   'content_length': 268,
   'ieee article id': '10849561',
   'doc_index': 47,
   'ieee issue id': '10820123',
   'total_pages': 25,
   'source': '..\\data\\pdf_files\\AgenticAI.pdf',
   'author': 'Unknown Author',
   'moddate': '2025-01-29T10:28:02-

In [14]:
rag_retriever.retrieve("Trends WITHIN LEGAL TASKS AND REAL-LIFE IMPLEMENTATION")

Retrieveing documents for query: 'Trends WITHIN LEGAL TASKS AND REAL-LIFE IMPLEMENTATION'
Top K :5,Score threshold :0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.76it/s]

Generated embeddings with shape:(1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_2fd14329_176',
  'content': 'judges, and lawyers, and facilitate automated resolution and\ndocument generation processes within the legal domain.\nThis endeavour involves defining and extracting legal tasks\nand identifying trends and uses of LLMs in the legal\ndomain.\nThe rest of the article is structured as follows: Section II\npresents the background and related work, and Section III\ndescribes the research method. Section IV reports a bib-\nliometric analysis of the literature reviewed, Section V\nprovides the overview of the models and approaches, whereas\nSection VI the trends within legal tasks and real-life\nimplementations. Section VII states the conclusions, and\nSection VIII discusses the implications of our findings for\nfuture research.\nII. BACKGROUND AND RELATED WORK\nIn this section, the landscape of legal AI is thoroughly\nexamined in various studies, highlighting its diverse appli-\ncations and advancements. The application of AI in the\nlegal sector has

### Intergration Vectordb Context pipeline With LLM output 

In [43]:
from langchain_groq import ChatGroq
import os
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv()


groq_api_key = os.environ.get("GROQ_API_KEY")

llm=ChatGroq(groq_api_key = groq_api_key,model_name="llama-3.3-70b-versatile",temperature=0.1,max_tokens= 1024)

def rag_simple(query, retriever, LLm ,top_k=3):
    retrieved_docs = retriever.retrieve(query,top_k=top_k)
    
    context = "\n\n".join([doc['content'] for doc in retrieved_docs]) if retrieved_docs else ""
   
    if not context:
        return " No relevant context found to answer the question."
    
    
    prompt = f""" use the following context to answer the question concisely.
        context:
        {context}

        Question: {query}

        Answer:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        return f"Error during LLM invocation: {e}"

In [44]:
answer =  rag_simple("Trends WITHIN LEGAL TASKS AND REAL-LIFE IMPLEMENTATION",rag_retriever,llm)
print(answer)

Retrieveing documents for query: 'Trends WITHIN LEGAL TASKS AND REAL-LIFE IMPLEMENTATION'
Top K :3,Score threshold :0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.56it/s]

Generated embeddings with shape:(1, 384)
Retrieved 3 documents (after filtering)


Section VI of the article discusses the trends within legal tasks and real-life implementations.
